https://www.analyticsvidhya.com/blog/2022/03/audio-denoiser-a-speech-enhancement-deep-learning-model/

https://towardsdatascience.com/40-open-source-audio-datasets-for-ml-59dc39d48f06

In [4]:
import numpy as np
from scipy.io import wavfile
from scipy.signal import spectrogram

def wiener_filter(input_file, noise_file, output_file):
    # Read input audio file
    fs, input_data = wavfile.read(input_file)

    # Read noise audio file
    _, noise_data = wavfile.read(noise_file)

    # Convert audio data to float and ensure mono-channel
    input_data = input_data.astype(np.float64)
    if input_data.ndim > 1:
        input_data = np.mean(input_data, axis=1)
    noise_data = noise_data.astype(np.float64)
    if noise_data.ndim > 1:
        noise_data = np.mean(noise_data, axis=1)

    # Compute spectrograms of input and noise signals
    _, _, input_spec = spectrogram(input_data, fs=fs, nperseg=1024, noverlap=512, window='hamming')
    _, _, noise_spec = spectrogram(noise_data, fs=fs, nperseg=1024, noverlap=512, window='hamming')

    # Estimate power spectral density (PSD) of the desired signal
    input_psd = np.mean(np.abs(input_spec)**2, axis=1)

    # Estimate power spectral density (PSD) of the noise
    noise_psd = np.mean(np.abs(noise_spec)**2, axis=1)

    # Compute Wiener filter coefficients
    wiener_filter = input_psd / (input_psd + noise_psd)

    # Apply Wiener filter to input spectrogram
    filtered_spec = input_spec * wiener_filter[:, np.newaxis]

    # Inverse spectrogram to obtain filtered time-domain signal
    _, filtered_data = np.fft.irfft(filtered_spec, axis=1)

    # Convert filtered audio data to 16-bit integer and write to output file
    filtered_data = filtered_data.astype(np.int16)
    wavfile.write(output_file, fs, filtered_data)

# Usage example
input_file = 'mixkit-small-group-cheer-and-applause-518.wav'
noise_file = r'C:\Users\nafem\Desktop\Intelligent Dive\Noise Cancelation\Wiener Filter\noise.wav'
output_file = 'output.wav'
wiener_filter(input_file, noise_file, output_file)



FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\nafem\\Desktop\\Intelligent Dive\\Noise Cancelation\\Wiener Filter\\noise.wav'

# Importing Libraries 

In [1]:
import numpy as np
import librosa
import soundfile as sf
import tensorflow as tf

# Load the input audio file

In [2]:
input_file = 'mixkit-small-group-cheer-and-applause-518.wav'
audio, sr = librosa.load(input_file, sr=None)

# Split the audio into overlapping frames

In [3]:
frame_length = 1024
hop_length = 256
frames = librosa.util.frame(audio, frame_length=frame_length, hop_length=hop_length).T

# Denoiser Architecture 

In [4]:
class Denoiser(tf.keras.Model):
    def __init__(self):
        super(Denoiser, self).__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(None,)),
            tf.keras.layers.Reshape((-1, 1)),
            tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu'),
            tf.keras.layers.Conv1D(64, 3, padding='same', activation='relu', strides=2),
            tf.keras.layers.Conv1D(128, 3, padding='same', activation='relu', strides=2),
            tf.keras.layers.Conv1D(256, 3, padding='same', activation='relu', strides=2),
        ])
        
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Conv1DTranspose(256, 3, padding='same', activation='relu', strides=2),
            tf.keras.layers.Conv1DTranspose(128, 3, padding='same', activation='relu', strides=2),
            tf.keras.layers.Conv1DTranspose(64, 3, padding='same', activation='relu', strides=2),
            tf.keras.layers.Conv1DTranspose(1, 3, padding='same', activation='sigmoid'),
            tf.keras.layers.Reshape((-1,))
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [5]:
# Load the pre-trained model
model = Denoiser()

# Apply noise cancellation to each frame

In [6]:
processed_frames = []
for frame in frames:
    # Normalize the frame
    normalized_frame = frame / np.max(np.abs(frame))
    
    # Apply noise cancellation using the model
    processed_frame = model.predict(np.expand_dims(normalized_frame, axis=0))[0]
    
    # Denormalize the processed frame
    processed_frame *= np.max(np.abs(frame))    
    processed_frames.append(processed_frame)

1/1 [==============================] - 0s 11ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 20ms/step


# Adjust the frames to align them

In [7]:
adjusted_frames = []
for i, frame in enumerate(processed_frames):
    start = i * hop_length
    end = start + frame_length
    adjusted_frame = np.zeros(frame_length)
    adjusted_frame[:len(frame)] = frame
    adjusted_frames.append(adjusted_frame)

# Combine the adjusted frames
processed_audio = np.concatenate(adjusted_frames)

# Save the processed audio to a file

In [8]:
output_file = 'processed_audio.wav'
sf.write(output_file, processed_audio, sr)